In [61]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
import yfinance as yf


In [62]:
returns_close = pd.read_csv('sp500_prices_close.csv', index_col=0, parse_dates=True)
returns_close = returns_close.dropna()
print(returns_close.head())


                    A       AAPL       ABBV        ABT       ACGL         ACN  \
Date                                                                            
2020-01-02  82.711060  72.538506  70.326820  78.555801  41.268997  192.514999   
2020-01-03  81.383072  71.833305  69.659286  77.598145  41.221451  192.194382   
2020-01-06  81.623665  72.405685  70.209023  78.004715  41.383106  190.939362   
2020-01-07  81.873848  72.065140  69.808502  77.571037  41.040779  186.816956   
2020-01-08  82.682205  73.224403  70.303268  77.887245  40.631893  187.183441   

                  ADBE         ADI        ADM         ADP  ...         WY  \
Date                                                       ...              
2020-01-02  334.429993  108.511078  39.126694  150.852661  ...  23.822412   
2020-01-03  331.809998  106.600868  39.050323  150.533798  ...  23.862932   
2020-01-06  333.709991  105.348450  38.744835  150.737503  ...  23.814302   
2020-01-07  333.390015  107.745201  38.278130  

# Engle-Granger test:

Assuming 2 time-series $X_{t}$ and $Y_{t}$ are I(1), they are cointegrated if there exist scalar $\beta$ such that :
 
$u_{t}= Y_{t}​− \beta X_{t}$ satisfies $u_{t}$~I(0)  

In [63]:
# Differentiation of a time-series
def diff(series, d=1):
    """Return Δ^d series (d-th difference)."""
    out = series.copy()
    for _ in range(d):
        out = out.diff()
    return out.dropna()

In [64]:
# ADF test
def adf_test(series, max_lags=None, ic='AIC', trend='c'):
    """
    trend: 'n' (no const), 'c' (const), 'ct' (const+trend), 'ctt' (const+trend+quad)
    ic:    'AIC' or 'BIC' for autolag selection
    """
    series = series.dropna()
    res = adfuller(series, maxlag=max_lags, regression=trend, autolag=ic)
    out = {
        'ADF stat': res[0],
        'p-value':  res[1],
        '#lags':    res[2],
        '#obs':     res[3],
        'crit':     res[4],   # dict with 1%, 5%, 10%
        'icbest':   res[5] if len(res) > 5 else None
    }
    return out

In [65]:
#checks the order of integration of a time-series
def is_Id(series, d=1, adf_trend='c', ic='AIC'):
    """Quick check: not stationary at level but stationary after one diff."""
    r0 = adf_test(series, trend=adf_trend, ic=ic)['p-value']
    r1 = adf_test(diff(series, d=d), trend='n', ic=ic)['p-value']
    return (r0 > 0.05) and (r1 < 0.05)

In [66]:
print(adf_test(diff(returns_close['AAPL']), trend='c'))

{'ADF stat': -35.938708244717134, 'p-value': 0.0, '#lags': 0, '#obs': 1256, 'crit': {'1%': -3.4355671297788666, '5%': -2.8638438984080117, '10%': -2.5679966213893057}, 'icbest': 5928.407008002422}


In [67]:
print(is_Id(returns_close['AAPL'],d=0))
print(is_Id(returns_close['AAPL'],d=1))

False
True


In [ ]:
def engle_granger(X,Y, p_val=0.05):
    if not (is_Id(X,d=1) and is_Id(Y,d=1)):
        return None
    else:
        score, p_value, _ = coint(X, Y)

        if p_value < p_val:
            X_const = sm.add_constant(X)
            model = sm.OLS(Y, X_const).fit()
            beta = model.params[1]  # slope coefficient
            alpha = model.params[0]  # intercept
            
            return beta, alpha, p_value, score
        else:
            return None

In [85]:
print(engle_granger(returns_close['AAPL'], returns_close['MSFT'], p_val=0.05))

None


# Testing values that are cointegrated in the SP500

In [ ]:
def cointegrated_pairs(df, p_val=0.05):
    pairs=[]
    for i in range(df.shape[1]):
        for j in range(i+1,df.shape[1]):
            if i!=j:
                    X = df.iloc[:,i]
                    Y = df.iloc[:,j]
                    if not engle_granger(X,Y, p_val=p_val) is None: 
                        beta, alpha, p_value, score = engle_granger(X,Y, p_val=p_val)
                        if p_value < p_val:
                            print(f"Cointegrated pair: {df.columns[i]} and {df.columns[j]} with p-value {p_value}")
                            pairs.append((df.columns[i], df.columns[j], beta, alpha, p_value, score))
    return pairs

In [ ]:
print(cointegrated_pairs(returns_close, p_val=0.05))

C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # slope coefficient
C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]  # intercept


Cointegrated pair: A and CFG with p-value 0.04014992138114454


C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # slope coefficient
C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]  # intercept


Cointegrated pair: A and F with p-value 0.047250515591776523


C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # slope coefficient
C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]  # intercept


...

Cointegrated pair: AAPL and PPL with p-value 0.026700259818609122


C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # slope coefficient
C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]  # intercept


Cointegrated pair: AAPL and RCL with p-value 0.018226103398567266


C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # slope coefficient
C:\Users\hando\AppData\Local\Temp\ipykernel_21124\3589658198.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]  # intercept


Cointegrated pair: AAPL and SPG with p-value 0.02409386405140318
